# GRAVITATIONAL LENS SDSSJ0819+5356

#### Stars study with Imaphot + Lens study with Imaphot

In [ ]:
# Packages
import utilfunctions as uf
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
from matplotlib import ticker
from astropy.nddata.utils import Cutout2D
import functions
import cupy as cp
from cupyx.scipy.ndimage import minimum_filter
import numpy as np
from skimage import exposure
from astropy.wcs import WCS
import psycopg2
import os
from astropy.io import fits
import io
from astropy.wcs import WCS
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
from matplotlib import ticker
from itertools import combinations

In [ ]:
# Paths of the gravitational lens
images_paths_QHY, images_paths_iKon, images_paths_all = uf.images_paths("SDSSJ0819+5356", directory_path = "work/red")

In [ ]:
# Components coordinates
coordsA=[124.998243440, 53.940423310]#A
coordsB=[125.00007152, 53.93966377] #B ra_decB = w_image.pixel_to_world( xA+3.367/0.507, yA+2.226/0.507)
coordsG=[124.999185780, 53.940057240] #G

In [ ]:
# Plot Lens
with fits.open(images_paths_iKon[100], memmap=True) as hdu:

    image, w_image, header, dateob, filter, fwhms, sky,EXPT1 = uf.getinfo(hdu)
    #print(header)
    xA, yA = uf.radec_to_xy(coordsA, w_image)
    xB, yB = uf.radec_to_xy(coordsB, w_image)
    xG, yG = uf.radec_to_xy(coordsG, w_image) 
    #ra_decB = w_image.pixel_to_world( xA+3.367/0.507, yA+2.226/0.507)
    #xB, yB = uf.radec_to_xy([ra_decB.ra.deg, ra_decB.dec.deg], w_image)
    normalized_image = exposure.rescale_intensity(image.get(), in_range='image', out_range=(0, 1))
    plt.imshow(normalized_image, vmin=0, vmax=0.015)

    plt.plot( xA, yA, "x", mec='g', ms=15)#A
    plt.plot( xA+3.367/0.507, yA+2.226/0.507, "x", mec='r', ms=15)#B
    plt.plot( xB, yB, "x", mec='b', ms=15)#A
    plt.plot( xG, yG, "x", mec='w', ms=15)#G
    plt.plot( xA+1.980/0.507, yA+1.348/0.507, "x", mec='w', ms=15)#G

    plt.xlim( xA+20, xA-20)
    plt.ylim( yA+20, yA-20)

    ra_decB = w_image.pixel_to_world( xA+3.367/0.507, yA+2.226/0.507)
    print(ra_decB,  xG, yG)
    xB, yB = uf.radec_to_xy([ra_decB.ra.deg, ra_decB.dec.deg], w_image)


# ________________________________________________________________________________________
## STARS STUDY

In [ ]:
coords = cp.asarray([[124.998243440, 53.940423310],[124.999185780, 53.940057240],[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.010332700, 53.927527680], [125.024931110, 53.921033100], [125.023014140, 53.950554040]])#[[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.012248390, 53.931427250], [125.010332700, 53.927527680], [124.985519970, 53.943565380], [125.024931110, 53.921033100], [124.985143180, 53.950553540], [125.023014140, 53.950554040]]) # D, E, F, G, H, I, J, K
print(len(coords))
stars=["A", "B", "D", "E", "G", "I", "K"]
with fits.open(images_paths_iKon[100], memmap=True) as hdu:
    image, w_image, header, dateob, filter, fwhms, sky,EXPT1 = uf.getinfo(hdu)
    x=[]
    y=[]
    for r,i in enumerate(coords):
        
        x0, y0 = uf.radec_to_xy(i.get(), w_image)
        x.append(x0)
        y.append(y0)

plt.imshow(image.get(), vmin=200, vmax=1000, cmap='gray' )
for j, k  in enumerate(x):
    if j == 0:
        plt.text(k-4, y[j]-4, stars[j],  color='white')
    elif j == 5:
        plt.text(k+10, y[j]+10, stars[j],  color='white')
    else:
        plt.text(k+15, y[j]+15, stars[j],  color='white')
height, width = image.shape
plt.annotate('', xy=(425, 1275), xytext=(425, 1200),arrowprops=dict(facecolor='white', edgecolor='white', arrowstyle='<-'),color='white')
plt.annotate('', xy=(422, 1272), xytext=(500, 1272),arrowprops=dict(facecolor='white', edgecolor='white', arrowstyle='<-'),color='white')
plt.annotate('N', xy=(425, 1200), xytext=(-5, 2),  textcoords='offset points', color='white', fontsize=10)
plt.annotate('E', xy=(500, 1272), xytext=( -8, -3),  textcoords='offset points', color='white', fontsize=10)
# Plot double horizontal arrows at position (800, 1275)
plt.annotate(
    '', xy=(850, 1275), xytext=(800, 1275),
    arrowprops=dict(facecolor='white', edgecolor='white', arrowstyle='<|-|>')
)

# Annotate with text "25.5" over the arrows
plt.annotate(
    '25.5"', xy=(820, 1275), xytext=(0, 5),
    textcoords='offset points', color='white', fontsize=10,
    ha='center', va='bottom'
)
plt.xlim(900, 400)
plt.ylim(1300, 800)

plt.axis('off')

plt.savefig('SDSSJ0819+5356_field.png')

In [ ]:
# Connect to DB
conn = psycopg2.connect(database="astrodb",
                        host="10.0.210.30",
                        user="read_only",
                        password="read_only",
                        port="5433")

# MODIFY these parameters (object 2)
# MODIFY these parameters
cursor = conn.cursor()
start_date = datetime.datetime(2023,1, 1,0,0,0)
end_date = datetime.datetime.now()

tolerance_small = 1/3600 # This is the tolerance to find objects in imaphot.
tolerance_big = 0.5 # This is used to pre-select relevant images on imastats. It should be approx the FOV of the images.
fluxstars={}

for i, c in enumerate(coords):
    ra = float(c[0])
    dec = float(c[1].get())
    cursor = conn.cursor()
    search_tuple = (ra, dec, tolerance_big, start_date, end_date, ra, dec, tolerance_small)
    cursor.execute("""SELECT imaphot.imageid, imastats.dateobs, imastats.header -> 'JD-OBS' AS jdobs, imaphot.ra, imaphot.dec, imaphot.flux, imaphot.dflux, CAST(imastats.header -> 'ZP' AS real) as zp, imastats.exptime, imastats.filter FROM imaphot JOIN imastats ON imaphot.imageid = imastats.imageid WHERE imaphot.imageid IN ( SELECT imageid FROM imastats WHERE q3c_radial_query(centralra, centraldec, %s, %s, %s) AND dateobs> %s AND dateobs< %s) AND q3c_radial_query(ra, dec, %s, %s, %s);""", search_tuple)
    data=cursor.fetchall()
    df = pd.DataFrame(data)
    df.columns = ["IMAGEID", "DATEOBS", "JD-OBS", "RA", "DEC", "FLUX", "DFLUX", "ZP", "EXPTIME", "FILTER"]
    df["MAG"] = df["ZP"]-2.5*np.log10(df["FLUX"]/df["EXPTIME"])
    cursor.close()
    fluxstars[stars[i]] = df

In [ ]:
dfs = []
dfs2 = []
for i in fluxstars:
    df_subset = fluxstars[i][['IMAGEID', "JD-OBS", "FILTER", 'FLUX']]
    df_subset2 = fluxstars[i][['IMAGEID', "JD-OBS", "FILTER", "DFLUX"]]
    dfs.append(df_subset)
    dfs2.append(df_subset2)
    
merged_df = dfs[0]
errormerged_df = dfs2[0]
for i, df in enumerate(dfs[1:]):
    merged_df = pd.merge(merged_df, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ stars[i],"_"+stars[i+1]))
# merged_df.rename(columns={"FLUX": "FLUX_"+stars[-1]}, inplace=True)
    if i == len(dfs) - 2:
        merged_df.rename(columns={"FLUX": "FLUX_" + stars[i+1]}, inplace=True)

for i, df in enumerate(dfs2[1:]):
    errormerged_df = pd.merge(errormerged_df, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ stars[i],"_"+stars[i+1]))
    if i == len(dfs2) - 2:
        errormerged_df.rename(columns={"DFLUX": "DFLUX_" + stars[i+1]}, inplace=True)
# errormerged_df.rename(columns={"DFLUX": "DFLUX_J"}, inplace=True)

for i in range(len(stars)):
    merged_df = merged_df[merged_df["FLUX_"+stars[i]].notna()] 
    errormerged_df = errormerged_df[errormerged_df["DFLUX_"+stars[i]].notna()]

print(merged_df) 



In [ ]:
unique_values = merged_df["FILTER"].unique()
fluxstars = {} # Dataframes flux star per filter
errorfluxstars = {} # Dataframes error_flux star per filter
dateobs = {}
for i, filters in enumerate(unique_values):
    fluxstars[f"{filters}"] = merged_df[merged_df["FILTER"] == filters].drop(merged_df.keys()[:3], axis=1)
    errorfluxstars[f"{filters}"] = errormerged_df[errormerged_df["FILTER"] == filters].drop(errormerged_df.keys()[:3], axis=1)
    dateobs[f"{filters}"] = merged_df[merged_df["FILTER"] == filters]["JD-OBS"]
print(unique_values)

In [ ]:
diff_phot, diff_phot_names, diff_phot_dates = star_diff_phot(fluxstars, dateobs, unique_values, stars)
error, errordiff_phot_names = error_diff_phot(fluxstars, errorfluxstars, unique_values, stars)

In [ ]:
¿a=[250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250,250]
for o, t in enumerate(diff_phot):
    print(len(diff_phot[t]), len(diff_phot[t]["filter_Lum"]))
    
    figure, axis = plt.subplots(len(diff_phot[t]), len(diff_phot[t]["filter_Lum"]),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(diff_phot[t]):
        for j, phot in enumerate(diff_phot[t][filtre]):
        
            axis[i][j].errorbar(diff_phot_dates[t]['filter_' + unique_values[i]][j]-2460000.0, phot, yerr=np.std((phot[np.logical_not(np.isnan(phot))])), elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = {np.std((phot[np.logical_not(np.isnan(phot))]))}")
            axis[i][j].legend()
            #axis[i][j].set_ylim([np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])-10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))]),np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])+10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])])
            axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
            axis[i][j].set_title("Filter "+unique_values[i]+ " star " + diff_phot_names[t]['filter_' + unique_values[i]][j][0]+ " and ref. star " + diff_phot_names[t]['filter_' + unique_values[i]][j][1:] )
    
    plt.show()
    #plt.savefig('diff_phot_stars_astroDB'+str(t)+'.png')

In [ ]:
import copy
diff_phot_dates2 = copy.deepcopy(diff_phot_dates)
combosbonesfiltre={}
combosbonesfiltrenom={}
for o, t in enumerate(diff_phot):
    std2=[]
    stdpercentile2=[]
    combos2sorted=[]
    minfiltre={}
    minfiltrenom={}
    for i, filters in enumerate(diff_phot[t]):
        if i>=0:
            stdinter=[]
            stdpercentileinter=[]
            #count=0
            for j, combo in enumerate(diff_phot[t][filters]):
                deletepositions2=[]
                # for o, l in enumerate(np.logical_not(np.isnan(combo))):
                    # if l == np.bool_(False):
                        # diff_phot_dates2[t][filters][j] = np.delete(diff_phot_dates2[t][filters][j], o)

                combo=(combo)[np.logical_not(np.isnan(combo))]    

                for k in combo:
                    if k < np.quantile(combo, 0.1) or k > np.quantile(combo, 0.9):     
                        deletepositions2.append(np.where(combo == k)[0][0])  
                combo = np.delete(combo, deletepositions2)
                # diff_phot_dates2[t][filters][j] = np.delete(diff_phot_dates2[t][filters][j], deletepositions2)
                
                stdinter.append(np.std(combo))
                stdpercentileinter.append(np.std(combo))
                #count+=1
            sortered = sorted(zip(stdpercentileinter,stdinter,diff_phot_names[t][filters]))

            std2.append([x[1] for x in sortered])
            stdpercentile2.append([x[0] for x in sortered])
            combos2sorted.append([x[2] for x in sortered])
            minfiltre[filters] = ([x[0] for x in sortered][0])
            minfiltrenom[filters] = ([x[2] for x in sortered][0])

    # figure, axis = plt.subplots(1, len(stdpercentile2),figsize=(60, 10))
    # figure.tight_layout(pad=3.5)
    combosbonesfiltre[t] = minfiltre
    combosbonesfiltrenom[t] = minfiltrenom
#     for i in range(len(stdpercentile2)):

#         #bars=axis[i].bar(combos3sorted[i], stdpercentile[i])
#         roundvalues=[round(val, 4) for val in stdpercentile2[i]]
#         axis[i].bar_label(axis[i].bar(combos2sorted[i], stdpercentile2[i]), labels = roundvalues, rotation=90)
#         axis[i].set(xlabel='Star groups', ylabel='Standard deviation')
#         axis[i].set_title("Filter "+ unique_values[i])
#         axis[i].tick_params(axis='x', rotation=90)
        
    #plt.savefig('standarddeviation_astroDB'+str(t)+'.png')

In [ ]:
for i, r in enumerate(combosbonesfiltre):
    print(r)
    for j, l in enumerate(combosbonesfiltre[r]):
        print(l)
        print(combosbonesfiltrenom[r][l],combosbonesfiltre[r][l])

# __________________________________________________

## LENS STUDY

In [ ]:
# Connect to DB
conn = psycopg2.connect(database="astrodb",
                        host="10.0.210.30",
                        user="read_only",
                        password="read_only",
                        port="5433")

# MODIFY these parameters
cursor = conn.cursor()
start_date = datetime.datetime(2023,1, 1,0,0,0)
end_date = datetime.datetime.now()
# D, E, F[125.012248390, 53.931427250], G, H[124.985519970, 53.943565380], I, J[124.985143180, 53.950553540], K
coordsstars = cp.asarray([[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.010332700, 53.927527680], [125.024931110, 53.921033100], [125.023014140, 53.950554040]]) # D, E, G, I, K
coordslens = cp.asarray([[124.998243440, 53.940423310],[124.999185780, 53.940057240]]) # A,  G ---B,,[125.00007152, 53.93966377]

stars=["D", "E", "G", "I", "K"]
lens=["A", "GB"]#---"B",

tolerance_small = 1/3600 # This is the tolerance to find objects in imaphot.
tolerance_big = 0.5 # This is used to pre-select relevant images on imastats. It should be approx the FOV of the images.


# STARS
fluxstars={}
for i, c in enumerate(coordsstars):
    ra = float(c[0].get())
    dec = float(c[1].get())
    cursor = conn.cursor()
    search_tuple = (ra, dec, tolerance_big, start_date, end_date, ra, dec, tolerance_small)
    cursor.execute("""SELECT imaphot.imageid, imastats.dateobs, imastats.header -> 'JD-OBS' AS jdobs, imaphot.ra, imaphot.dec, imaphot.flux, imaphot.dflux, CAST(imastats.header -> 'ZP' AS real) as zp, imastats.exptime, imastats.filter FROM imaphot JOIN imastats ON imaphot.imageid = imastats.imageid WHERE imaphot.imageid IN ( SELECT imageid FROM imastats WHERE q3c_radial_query(centralra, centraldec, %s, %s, %s) AND dateobs> %s AND dateobs< %s) AND q3c_radial_query(ra, dec, %s, %s, %s);""", search_tuple)
    data=cursor.fetchall()
    df = pd.DataFrame(data)
    df.columns = ["IMAGEID", "DATEOBS", "JD-OBS", "RA", "DEC", "FLUX", "DFLUX", "ZP", "EXPTIME", "FILTER"]
    df["MAG"] = df["ZP"]-2.5*np.log10(df["FLUX"]/df["EXPTIME"])
    cursor.close()
    fluxstars[stars[i]] = df

# COMPONENTS+Galaxy
fluxcomp={}
for i, c in enumerate(coordslens):
    ra = float(c[0].get())
    dec = float(c[1].get())
    cursor = conn.cursor()
    search_tuple = (ra, dec, tolerance_big, start_date, end_date, ra, dec, tolerance_small)
    cursor.execute("""SELECT imaphot.imageid, imastats.dateobs, imastats.header -> 'JD-OBS' AS jdobs, imaphot.ra, imaphot.dec, imaphot.flux, imaphot.dflux, CAST(imastats.header -> 'ZP' AS real) as zp, imastats.exptime, imastats.filter FROM imaphot JOIN imastats ON imaphot.imageid = imastats.imageid WHERE imaphot.imageid IN ( SELECT imageid FROM imastats WHERE q3c_radial_query(centralra, centraldec, %s, %s, %s) AND dateobs> %s AND dateobs< %s) AND q3c_radial_query(ra, dec, %s, %s, %s);""", search_tuple)
    data=cursor.fetchall()
    df = pd.DataFrame(data)
    df.columns = ["IMAGEID", "DATEOBS", "JD-OBS", "RA", "DEC", "FLUX", "DFLUX", "ZP", "EXPTIME", "FILTER"]
    df["MAG"] = df["ZP"]-2.5*np.log10(df["FLUX"]/df["EXPTIME"])
    cursor.close()
    fluxcomp[lens[i]] = df

In [ ]:
# STARS
dfs = []
dfs2 = []
for i in fluxstars:
    df_subset = fluxstars[i][['IMAGEID', "JD-OBS", "FILTER", 'FLUX']]
    df_subset2 = fluxstars[i][['IMAGEID', "JD-OBS", "FILTER", "DFLUX"]]
    dfs.append(df_subset)
    dfs2.append(df_subset2)
    
merged_df = dfs[0]
errormerged_df = dfs2[0]
for i, df in enumerate(dfs[1:]):
    merged_df = pd.merge(merged_df, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ stars[i],"_"+stars[i+1]))
# merged_df.rename(columns={"FLUX": "FLUX_"+stars[-1]}, inplace=True)
    if i == len(dfs) - 2:
        merged_df.rename(columns={"FLUX": "FLUX_" + stars[i+1]}, inplace=True)

for i, df in enumerate(dfs2[1:]):
    errormerged_df = pd.merge(errormerged_df, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ stars[i],"_"+stars[i+1]))
    if i == len(dfs2) - 2:
        errormerged_df.rename(columns={"DFLUX": "DFLUX_" + stars[i+1]}, inplace=True)
# errormerged_df.rename(columns={"DFLUX": "DFLUX_J"}, inplace=True)

for i in range(len(stars)):
    merged_df = merged_df[merged_df["FLUX_"+stars[i]].notna()] 
    errormerged_df = errormerged_df[errormerged_df["DFLUX_"+stars[i]].notna()]

#print(merged_df) 

# COMPONENTS + Galaxy
dfs = []
dfs2 = []
for i in fluxcomp:
    df_subset = fluxcomp[i][['IMAGEID', "JD-OBS", "FILTER", 'FLUX']]
    df_subset2 = fluxcomp[i][['IMAGEID', "JD-OBS", "FILTER", "DFLUX"]]
    dfs.append(df_subset)
    dfs2.append(df_subset2)
    
merged_dfcomp = dfs[1]
errormerged_dfcomp = dfs2[1]
#for i, df in enumerate(dfs[1:]):
#    merged_dfcomp = pd.merge(merged_dfcomp, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ lens[i],"_"+lens[i+1]))
# merged_df.rename(columns={"FLUX": "FLUX_"+stars[-1]}, inplace=True)
#    if i == len(dfs) - 2:
#        merged_dfcomp.rename(columns={"FLUX": "FLUX_" + lens[i+1]}, inplace=True)

#for i, df in enumerate(dfs2[1:]):
#    errormerged_dfcomp = pd.merge(errormerged_dfcomp, df, on=['IMAGEID',"JD-OBS", "FILTER"], how='outer', suffixes=("_"+ lens[i],"_"+lens[i+1]))
#    if i == len(dfs2) - 2:
#        errormerged_dfcomp.rename(columns={"DFLUX": "DFLUX_" + lens[i+1]}, inplace=True)
# errormerged_df.rename(columns={"DFLUX": "DFLUX_J"}, inplace=True)
#print(merged_dfcomp) 
#for i in range(len(lens)):
merged_dfcomp = merged_dfcomp[merged_dfcomp["FLUX"].notna()] #_"+lens[i]
errormerged_dfcomp = errormerged_dfcomp[errormerged_dfcomp["DFLUX"].notna()]
#print(merged_dfcomp) 

merged_dftot = pd.merge(merged_df, merged_dfcomp, on=['IMAGEID',"JD-OBS", "FILTER"], how='inner')
errormerged_dftot = pd.merge(errormerged_df, errormerged_dfcomp, on=['IMAGEID',"JD-OBS", "FILTER"], how='inner')
#print(merged_dftot) 
print(merged_dftot)
#for i in range(len(nomc)):    
merged_dftot = np.split(merged_dftot, [3, 3+len(stars)], axis=1 )
errormerged_dftot = np.split(errormerged_dftot, [3, 3+len(stars)], axis=1 )

starsflux = pd.concat([merged_dftot[0],merged_dftot[1]], axis=1)
componentsflux = pd.concat([merged_dftot[0],merged_dftot[2]], axis=1)
estarsflux = pd.concat([errormerged_dftot[0],errormerged_dftot[1]], axis=1)
ecomponentsflux = pd.concat([errormerged_dftot[0],errormerged_dftot[2]], axis=1)


In [ ]:
unique_values = merged_dftot[0]["FILTER"].unique()
fluxstars = {} # Dataframes flux star per filter
errorfluxstars = {} # Dataframes error_flux star per filter
fluxcomponents = {} # Dataframes flux components per filter
errorfluxcomponents = {} # Dataframes error_flux components per filter
dateobs = {}
ids ={}
for i, filters in enumerate(unique_values):
    fluxstars[f"{filters}"] = starsflux[starsflux["FILTER"] == filters].drop(starsflux.keys()[:3], axis=1)
    errorfluxstars[f"{filters}"] = estarsflux[estarsflux["FILTER"] == filters].drop(starsflux.keys()[:3], axis=1)
    fluxcomponents[f"{filters}"] = componentsflux[componentsflux["FILTER"] == filters].drop(componentsflux.keys()[:3], axis=1)
    errorfluxcomponents[f"{filters}"] = ecomponentsflux[ecomponentsflux["FILTER"] == filters].drop(componentsflux.keys()[:3], axis=1)
    dateobs[f"{filters}"] = merged_dftot[0][merged_dftot[0]["FILTER"] == filters]["JD-OBS"]
    ids[f"{filters}"] = merged_dftot[0][merged_dftot[0]["FILTER"] == filters]["IMAGEID"]
print(unique_values) 

In [ ]:
difmag = -2.5*(fluxcomponents["Lum"]["FLUX"]/fluxstars["Lum"]["FLUX_G"]).apply(lambda x: np.log10(x))
plt.plot(dateobs["Lum"],difmag,linewidth=0, marker=".")
plt.show()

In [ ]:
# Function for differential photometry between stars with the flux and the dateobs in arrays
def star_diff_phot(fluxstars, dateobs, diferent_filters, starsname = False): 
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # dateobs: array separated by filters, each date corresponds to a different image.
    # starsname: array with names of the stars, default the stars are named with numbers.
    # diferent_filters: array with the filters
    
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[diferent_filters[0]].keys()))]
    
    diff_phot = {} # Diff. phot values
    diff_phot_names = {} # Diff. phot star combinations names
    diff_phot_dates = {} # Dates for each diff. phot value
    
    for num_stars in range(1, len(starsname)): # Differential photometry using different numbers of stars like reference stars
        diff_phot[num_stars] = {} 
        diff_phot_names[num_stars] = {}
        diff_phot_dates[num_stars] = {}
                           
        for i, flux_star in enumerate(fluxstars):  # Separated for each filter
            diff_phot[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_names[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_dates[num_stars]['filter_' + diferent_filters[i]] = []  
            
            for j, star_flux in enumerate(fluxstars[flux_star]): # Differential photometry of one star in front of the rest
                for combo in combinations(np.delete(fluxstars[flux_star].keys(), j), num_stars): # Combinations of the stars used like reference stars
                    diffphot = -2.5 * (sum(fluxstars[flux_star][k] for k in combo) / fluxstars[flux_star][star_flux]).apply(lambda x: np.log10(x))
                    diff_phot[num_stars]['filter_' + diferent_filters[i]].append(diffphot)
                    diff_phot_names[num_stars]['filter_' + diferent_filters[i]].append(starsname[j] + ''.join(k[-1] for k in combo))
                    diff_phot_dates[num_stars]['filter_' + diferent_filters[i]].append(np.float64(dateobs[flux_star]))

    return diff_phot, diff_phot_names, diff_phot_dates

In [ ]:
def error_diff_phot(fluxstars, errorfluxstars, diferent_filters, starsname = False):
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[0]))]
    
    error = {}
    errordiff_phot_names = {}
    for num_stars in range(1, len(starsname)): # Differential photometry using different numbers of stars like reference stars
        error[num_stars] = {} 
        errordiff_phot_names[num_stars] = {}                  
        for i, flux_star in enumerate(fluxstars):  # Separated for each filter
            error[num_stars]['filter_' + diferent_filters[i]] = []
            errordiff_phot_names[num_stars]['filter_' + diferent_filters[i]] = []
            for j, star_flux in enumerate(fluxstars[flux_star]): # Differential photometry of one star in front of the rest
                for combo in combinations(np.delete(fluxstars[flux_star].keys(), j), num_stars): # Combinations of the stars used like reference stars
                    errordiffphot = 2.5 * np.log10(np.e) * (sum((errorfluxstars[flux_star]["D"+k]**2) for k in combo)/((sum(fluxstars[flux_star][k] for k in combo))**2) + (errorfluxstars[flux_star]["D"+star_flux]/fluxstars[flux_star][star_flux])**2).apply(lambda x: np.sqrt(x)) 
                    error[num_stars]['filter_' + diferent_filters[i]].append(errordiffphot)
                    errordiff_phot_names[num_stars]['filter_' + diferent_filters[i]].append(starsname[j] + ''.join(k[-1] for k in combo))

    return error, errordiff_phot_names 

In [ ]:
# Function for differential photometry between the components of the lens and the stars with the flux and the dateobs in arrays
def component_diff_phot(fluxstars, fluxcomp, dateobs, diferent_filters, starsname = False, compname = False): 
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # fluxcomp: array separated by filters, in each filter the flux values of each component in different arrays, for a component each value of flux corresponds to a different image.
    # dateobs: array separated by filters, each date corresponds to a different image.
    # starsname: array with names of the stars, default the stars are named with numbers.
    # compname: array with names of the components, default the components are named with numbers.
    # diferent_filters: array with the filters
    # count=0
    # count2=0
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[diferent_filters[0]].keys()))]
        
    if compname == False: # Components named with numbers
        compname = [str(i) for i in range(len(fluxcomp[diferent_filters[0]].keys()))]
    
    diff_photA = {} # Diff. phot values
    diff_phot_namesA = {} # Diff. phot components and stars combinations names
    diff_phot_datesA = {} # Dates for each diff. phot value
    
    diff_photB = {} # Diff. phot values
    diff_phot_namesB = {} # Diff. phot components and stars combinations names
    diff_phot_datesB = {} # Dates for each diff. phot value
    
    for num_stars in range(1, len(starsname)+1): # Differential photometry using different numbers of stars like reference stars
        diff_photA[num_stars] = {} 
        diff_phot_namesA[num_stars] = {}
        diff_phot_datesA[num_stars] = {}
        
        diff_photB[num_stars] = {} 
        diff_phot_namesB[num_stars] = {}
        diff_phot_datesB[num_stars] = {}
                           
        for i, filters in enumerate(fluxstars):  # Separated for each filter
            diff_photA[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_namesA[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_datesA[num_stars]['filter_' + diferent_filters[i]] = []  
            
            diff_photB[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_namesB[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_datesB[num_stars]['filter_' + diferent_filters[i]] = [] 
            
            for j, comp_flux in enumerate(fluxcomp[filters]): # Differential photometry of one component in front of the stars
                for combo in combinations(fluxstars[filters].keys(), num_stars): # Combinations of the stars used like reference stars
                    if j==0:
                        diffphotA = -2.5 * (fluxcomp[filters][comp_flux] / sum(fluxstars[filters][k] for k in combo)).apply(lambda x: np.log10(x)) 
                        diff_photA[num_stars]['filter_' + diferent_filters[i]].append(np.array(diffphotA))
                        diff_phot_namesA[num_stars]['filter_' + diferent_filters[i]].append(compname[j] + ''.join(k[-1] for k in combo))
                        diff_phot_datesA[num_stars]['filter_' + diferent_filters[i]].append(np.float64(dateobs[filters]))
                        # if count==0:
                        #     print(diffphotA,np.float64(dateobs[filters]))
                        # count+=1
                    else:
                        
                        diffphotB = -2.5 * (fluxcomp[filters][comp_flux] / sum(fluxstars[filters][k] for k in combo)).apply(lambda x: np.log10(x)) 
                        diff_photB[num_stars]['filter_' + diferent_filters[i]].append(np.array(diffphotB))
                        diff_phot_namesB[num_stars]['filter_' + diferent_filters[i]].append(compname[j] + ''.join(k[-1] for k in combo))
                        diff_phot_datesB[num_stars]['filter_' + diferent_filters[i]].append(np.float64(dateobs[filters]))
                        # if count2==0:
                        #     print(diffphotB,np.float64(dateobs[filters]))
                        # count2+=1
    return diff_photA, diff_phot_namesA, diff_phot_datesA, diff_photB, diff_phot_namesB, diff_phot_datesB

In [ ]:
# Function for compute the scale factors for the σ of differential photometry between components and stars
def scalefactors(fluxstars, errorfluxstars, fluxcomp, errorfluxcomp, diferent_filters, starsname = False):
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # fluxcomp: array separated by filters, in each filter the flux values of each component in different arrays, for a component each value of flux corresponds to a different image.
    # diferent_filters: array with the filters
    # starsname: array with names of the stars, default the stars are named with numbers.
    # compname: array with names of the components, default the components are named with numbers.
    # diferent_filters: array with the filters
    
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[diferent_filters[0]].keys()))]
    
    scale_factorsA = {} # Scale factors (SF) -> σ_comp = SF * σ_stars_after_plot
    scale_factorsB = {} 
    scale_factors_namesA = {} # To know which components and which stars have been used 
    scale_factors_namesB = {}
    
    for num_stars in range(1, len(starsname)): # Number of reference stars
        scale_factorsA[num_stars] = {}
        scale_factors_namesA[num_stars] = {}
        scale_factorsB[num_stars] = {}
        scale_factors_namesB[num_stars] = {}
        
        for i, flux_star in enumerate(fluxstars): # Separated for each filter
            scale_factorsA[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factors_namesA[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factorsB[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factors_namesB[num_stars]['filter_' + diferent_filters[i]] = []
            
            for j, comp_flux in enumerate(fluxcomp[flux_star]):
                for combo in combinations(fluxstars[flux_star].keys(), num_stars): # Implementation the sum of stars' fluxes
                    for l, s in enumerate(fluxstars[flux_star]): 
                        if s not in combo: # Stars no used in the combo to compute the σ_stars_from_flux_error needed for SF = σ_stars_from_flux_error/σ_comp_from_flux_error
                            # scale_factors[num_stars]['filter_' + diferent_filters[i]].append((fluxstars[flux_star][s] / fluxcomp[flux_star][comp_flux]) ** 2 * (
                            #             ((sum(fluxstars[flux_star][k] for k in combo))**2 * (errorfluxcomp[flux_star]["D"+comp_flux])**2 +
                            #              (fluxcomp[flux_star][comp_flux])**2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2) /
                            #             (fluxstars[flux_star][s]** 2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2 +
                            #              errorfluxstars[flux_star]["D"+s]** 2 * (sum(fluxstars[flux_star][k] for k in combo))**2)))
                            if j == 0: # A
                                scale_factorsA[num_stars]['filter_' + diferent_filters[i]].append((fluxstars[flux_star][s] / fluxcomp[flux_star][comp_flux]) ** 2 * (
                                        ((sum(fluxstars[flux_star][k] for k in combo))**2 * (errorfluxcomp[flux_star]["D"+comp_flux])**2 +
                                         (fluxcomp[flux_star][comp_flux])**2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2) /
                                        (fluxstars[flux_star][s]** 2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2 +
                                         errorfluxstars[flux_star]["D"+s]** 2 * (sum(fluxstars[flux_star][k] for k in combo))**2)))
                                scale_factors_namesA[num_stars]['filter_' + diferent_filters[i]].append("A" + starsname[l] + ''.join(k[-1] for k in combo))
                            else: # B
                                scale_factors_namesB[num_stars]['filter_' + diferent_filters[i]].append("B" + starsname[l] + ''.join(k[-1] for k in combo))   
                                scale_factorsB[num_stars]['filter_' + diferent_filters[i]].append((fluxstars[flux_star][s] / fluxcomp[flux_star][comp_flux]) ** 2 * (
                                        ((sum(fluxstars[flux_star][k] for k in combo))**2 * (errorfluxcomp[flux_star]["D"+comp_flux])**2 +
                                         (fluxcomp[flux_star][comp_flux])**2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2) /
                                        (fluxstars[flux_star][s]** 2 * (sum(errorfluxstars[flux_star]["D"+k] for k in combo))**2 +
                                         errorfluxstars[flux_star]["D"+s]** 2 * (sum(fluxstars[flux_star][k] for k in combo))**2)))
    return scale_factorsA, scale_factors_namesA, scale_factorsB, scale_factors_namesB

In [ ]:
compname = ["A","B"]
starsname = ["D", "E", "G", "I", "K"]
diff_phot, diff_phot_names, diff_phot_dates = star_diff_phot(fluxstars, dateobs, unique_values, starsname)
diff_phot_compA, diff_phot_names_compA, diff_phot_dates_compA, diff_phot_compB, diff_phot_names_compB, diff_phot_dates_compB = component_diff_phot(fluxstars, fluxcomponents, dateobs, unique_values, starsname, compname)
scale_factorsA, scale_factors_namesA, scale_factorsB, scale_factors_namesB = scalefactors(fluxstars, errorfluxstars, fluxcomponents, errorfluxcomponents, unique_values, starsname)
error, errordiff_phot_names = error_diff_phot(fluxstars, errorfluxstars, unique_values, starsname)

In [ ]:
a=[250,250,250,50]
for o, t in enumerate(diff_phot):
    print(len(diff_phot[t]), len(diff_phot[t]["filter_Lum"]))
    
    figure, axis = plt.subplots(len(diff_phot[t]), len(diff_phot[t]["filter_Lum"]),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(diff_phot[t]):
        for j, phot in enumerate(diff_phot[t][filtre]):
        
            axis[i][j].errorbar(diff_phot_dates[t]['filter_' + unique_values[i]][j]-2460000.0, phot, yerr=np.std((phot[np.logical_not(np.isnan(phot))])), elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = {np.std((phot[np.logical_not(np.isnan(phot))]))}")
            axis[i][j].legend()
            #axis[i][j].set_ylim([np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])-10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))]),np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])+10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])])
            axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
            axis[i][j].set_title("Filter "+unique_values[i]+ " star " + diff_phot_names[t]['filter_' + unique_values[i]][j][0]+ " and ref. star " + diff_phot_names[t]['filter_' + unique_values[i]][j][1:] )
    
    plt.show()
    #plt.savefig('diff_phot_stars_astroDB'+str(t)+'.png')

In [ ]:
# Extract outliers
def exctract_outliers(data, min_quantile, max_quantile):
    
    deletepositions=[]
    
    for k in data:
        if k < np.quantile(data, min_quantile) or k > np.quantile(data, max_quantile):                                    
            deletepositions.append(np.where(data == k)[0][0])
            
    return deletepositions

In [ ]:
def exctract_outliers2(data, time, min_quantile, max_quantile):
    deletepositions=[]
    
    df = pd.DataFrame({'X': time, 'Y': data})
    
    X_train, X_test, Y_train, Y_test = train_test_split(time.reshape(-1, 1), data, test_size=0.2, random_state=42)
    
    # Fit the model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, Y_train)

    # Predict on the training set
    df['Predicted'] = model.predict(time.reshape(-1, 1))
    
    # Calculate residuals
    df['Residuals'] = df['Y'] - df['Predicted']

    # Standardize residuals
    df['StdResiduals'] = (df['Residuals'] - df['Residuals'].mean()) / df['Residuals'].std()

    # Identify outliers using Z-score method (threshold |Z| > 2 for this example)
    outliers = df[np.abs(df['StdResiduals']) > 2]
    
    for k in outliers["Y"]:
        deletepositions.append(np.where(data == k)[0][0])
    # print(len(df['Predicted']), len(data), len(time))
    return deletepositions, df['Predicted']

In [ ]:
def __data_time_binning(data, time, mjd_ini, mjd_end, time_interval_hours, parameter, parameter_error):
    """
    Parameters
    ----------
    data: panda dataframe. Mandatory to have columns [time,parameter,parameter_error] i.e. ['MJD','MAG','eMAG']
    time: column time name 
    mjd_ini/mjd_end: MJD initial/final float
    time_interval_hours: time interval units hours float
    parameter: column parameter name to do stats string
    parameter_error: column parameter name with individual errors string
    
    
    Return
    ----------
    result_df: panda df [time+'_min', time+'_max', time+'_median',time+'std',time+'_n', parameter+'_median', parameter+'_std', parameter+'_n',time+'_central','e'+time]
    """    
    MJD_bins=int((mjd_end-mjd_ini)*24/time_interval_hours)
    
    x_median,y_median,n_x,n_y = __binXY(data[time],data[parameter],statistic='median',xbins=MJD_bins,xrange=None)
    # print(x_median,y_median,n_x,n_y)
    x_std,y_std,n_x,n_y = __binXY(data[time],data[parameter],statistic='std',xbins=MJD_bins,xrange=None)
    x_min,y_min,n_x,n_y = __binXY(data[time],data[parameter],statistic='min',xbins=MJD_bins,xrange=None)
    x_max,y_max,n_x,n_y = __binXY(data[time],data[parameter],statistic='max',xbins=MJD_bins,xrange=None)
    data_bin=pd.DataFrame({time+'_min': x_min, time+'_max': x_max, time+'_median': x_median, time+'_std': x_std, time+'_n': n_x, parameter+'_median': y_median, parameter+'_std': y_std, parameter+'_n': n_y})
    data_bin = data_bin.dropna() # dropping NaN lines
    data_bin['e'+parameter] = data_bin[parameter+'_std'] / np.sqrt(data_bin[parameter+'_n'])
    data_bin[time+'_central']=data_bin[time+'_min']+(data_bin[time+'_max']-data_bin[time+'_min'])/2
    data_bin['e'+time]=(data_bin[time+'_max']-data_bin[time+'_min'])/2
    return(data_bin)

In [ ]:
def __binXY(x,y,statistic='mean',xbins=10,xrange=None):
    """
    Finds statistical value of x and y values in each x bin. 
    Returns the same type of statistic for both x and y.
    See scipy.stats.binned_statistic() for options.
    
    Parameters
    ----------
    x : array
        x values.
    y : array
        y values.
    statistic : string or callable, optional
        See documentation for scipy.stats.binned_statistic(). Default is mean.
    xbins : int or sequence of scalars, optional
        If xbins is an integer, it is the number of equal bins within xrange.
        If xbins is an array, then it is the location of xbin edges, similar
        to definitions used by np.histogram. Default is 10 bins.
        All but the last (righthand-most) bin is half-open. In other words, if 
        bins is [1, 2, 3, 4], then the first bin is [1, 2) (including 1, but 
        excluding 2) and the second [2, 3). The last bin, however, is [3, 4], 
        which includes 4.    
        
    xrange : (float, float) or [(float, float)], optional
        The lower and upper range of the bins. If not provided, range is 
        simply (x.min(), x.max()). Values outside the range are ignored.
    
    Returns
    -------
    x_stat : array
        The x statistic (e.g. mean) in each bin. 
    y_stat : array
        The y statistic (e.g. mean) in each bin.       
    n_x : array of dtype int
        The count of x values in each bin.
    n_y : array of dtype int
        The count of y values in each bin.        
        """
    x_stat, xbin_edges, binnumber = stats.binned_statistic(x, x, 
                                 statistic=statistic, bins=xbins, range=xrange)
    #print(x_stat)
    y_stat, xbin_edges, binnumber = stats.binned_statistic(x, y, 
                                 statistic=statistic, bins=xbins, range=xrange)
    n_x, xbin_edges, binnumber = stats.binned_statistic(x, x, 
                                 statistic='count', bins=xbins, range=xrange)
    n_y, xbin_edges, binnumber = stats.binned_statistic(x, y, 
                                 statistic='count', bins=xbins, range=xrange)
            
    return x_stat, y_stat, n_x, n_y

In [ ]:
# PLOTS COMPONENTS + STARS + SCALE FACTORS, substract outliers and binning (improve it)
import copy
from matplotlib.ticker import MaxNLocator
import statistics as stat
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
a=[250,400,250,100]
for o, t in enumerate(scale_factors_namesA):
    figure, axis = plt.subplots(len(scale_factors_namesA[t]), int(len(scale_factors_namesA[t]["filter_Lum"])),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(scale_factors_namesA[t]):
        for j, l in enumerate(scale_factors_namesA[t][filtre]):
            for r, n in enumerate(diff_phot_names[t]['filter_' + unique_values[i]]):
                if n==l[1:]:
                    for g, phot in enumerate(diff_phot_compA[t]['filter_' + unique_values[i]]):
                        
                        if l[0]==diff_phot_names_compA[t]['filter_' + unique_values[i]][g][0] and  diff_phot_names_compA[t]['filter_' + unique_values[i]][g][1:]==n[1:]: 
                            
                            photA = copy.deepcopy(phot)
                            datesA = copy.deepcopy(diff_phot_dates_compA[t]['filter_' + unique_values[i]][g])
                            sfA = copy.deepcopy(scale_factorsA[t][filtre][j])
                            
                            deletepositions0 = exctract_outliers(data = photA,  min_quantile = 0.03, max_quantile = 0.97)
                            photA = np.delete(photA, deletepositions0)
                            datesA = np.delete(datesA, deletepositions0)
                            sfA = np.delete(sfA, deletepositions0)
                            
                            deletepositions, pred1 = exctract_outliers2(data = photA, time = datesA,  min_quantile = 0.03, max_quantile = 0.97)
                            # print(len(deletepositions), len(pred1), len(datesA), len(photA))
                            photA = np.delete(photA, deletepositions)
                            datesA = np.delete(datesA, deletepositions)
                            sfA = np.delete(sfA, deletepositions)
                            pred1 = np.delete(pred1, deletepositions)
                            
                            # print(len(pred1), len(datesA), len(photA))
                            
                            photB = copy.deepcopy(diff_phot_compB[t]['filter_' + unique_values[i]][g])
                            datesB = copy.deepcopy(diff_phot_dates_compB[t]['filter_' + unique_values[i]][g])
                            sfB = copy.deepcopy(scale_factorsB[t][filtre][j])
                              
                            deletepositions02 = exctract_outliers(data = diff_phot_compB[t]['filter_' + unique_values[i]][g],min_quantile = 0.03, max_quantile = 0.97)
                            photB = np.delete(photB, deletepositions02)
                            datesB = np.delete(datesB, deletepositions02)     
                            sfB = np.delete(sfB, deletepositions02)
                            
                            deletepositions2, pred2 = exctract_outliers2(data = photB, time = datesB, min_quantile = 0.03, max_quantile = 0.97)
                            # print(len(pred2), len(datesB), len(photB))                     
                            photB = np.delete(photB, deletepositions2)
                            datesB = np.delete(datesB, deletepositions2)     
                            sfB = np.delete(sfB, deletepositions2)  
                            pred2 = np.delete(pred2, deletepositions2)
                            # print(len(pred2), len(datesB), len(photB))
                            
                            deletepositions3=[]
                            for u, x in enumerate(sfA):
                                if x > (stat.mode(sfA)+stat.mode(sfA)/2):# or x < (stat.mode(sfA)-stat.mode(sfA)/2):
                                    deletepositions3.append(u)
                                                                       
                            photA = np.delete(photA, deletepositions3)
                            datesA = np.delete(datesA, deletepositions3)
                            sfA = np.delete(sfA, deletepositions3)
                            pred1 = np.delete(pred1, deletepositions3)

                            
                            deletepositions4=[]
                            for u, x in enumerate(sfB):
                                if x > (stat.mode(sfB)+stat.mode(sfB)/2):# or x < (stat.mode(sfB)-stat.mode(sfB)):
                                    deletepositions4.append(u)
                            photB = np.delete(photB, deletepositions4)
                            datesB = np.delete(datesB, deletepositions4)
                            sfB = np.delete(sfB, deletepositions4)
                            pred2 = np.delete(pred2, deletepositions4)

                        
                            err=diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]
                            #deletepositions5=[]
                            #for k in diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]:
                                #if k < np.quantile(err, 0.1) or k > np.quantile(err, 0.9):     
                                    #deletepositions5.append(np.where(err == k)[0][0]) 
                            
                            deletepositions5 = exctract_outliers(data = diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))], min_quantile = 0.1, max_quantile = 0.9)
                            
                            err = np.delete(err, deletepositions5)
                    
                            lista3= zip(datesA, photA, sfA*np.std(err))
                            # print(type(lista3))
                            lista4= zip(datesB, photB, sfB*np.std(err))
                            dfA = pd.DataFrame(lista3, columns=['MJD','MAG','eMAG'])
                            dfB = pd.DataFrame(lista4, columns=['MJD','MAG','eMAG'])
                                #print(dfA)
                                
                            data_binA = __data_time_binning(data=dfA, time='MJD', mjd_ini=min(datesA), mjd_end=max(datesA), time_interval_hours=24.0, parameter='MAG', parameter_error='eMAG')
                            data_binB = __data_time_binning(data=dfB, time='MJD', mjd_ini=min(datesB), mjd_end=max(datesB), time_interval_hours=24.0, parameter='MAG', parameter_error='eMAG')
                     
                            # datesA2, photA2, ephotA2 = bin_flux_data_julian(photA, datesA, 24)
                            # datesB2, photB2, ephotB2 = bin_flux_data_julian(photB, datesB, 24)
                            # datesA3, sfA2, ee = bin_flux_data_julian(sfA, datesA, 24)
                            # datesB3, sfB2, ee = bin_flux_data_julian(sfB, datesB, 24)
                            
                            if i == 1:
                                if j==0:
                                    finalA=data_binA["MAG_median"]
                                    finalB=data_binB["MAG_median"]
                                    finaldataA=data_binA["MJD_central"]
                                    finaldataB=data_binB["MJD_central"]
                                    
                            # for q,w in enumerate(ephotA2):
                            #     if w==0:
                            #         ephotA2[q]=sfA2[q]*np.std((err))
                            # for q,w in enumerate(ephotB2):
                            #     if w==0:
                            #         ephotB2[q]=sfB2[q]*np.std((err))  pred1
                            
                            axis[i][j].errorbar((datesA)-2460000.0, photA, yerr=sfA*np.std((err)), elinewidth=0.75, linewidth=0,   marker=".", label = f"Component A ($σA _i = \Gamma ^{2}_i $ x {np.std((err))})") # ({np.mean(sfA)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))
                            axis[i][j].errorbar((datesB)-2460000.0, photB, yerr=sfB*np.std((err)), elinewidth=0.75, linewidth=0,   marker=".", label = f"Component B ($σB _i = \Gamma ^{2}_i $ x {np.std((err))})") #({np.mean(sfB)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))
                            
                            axis[i][j].errorbar((data_binA["MJD_central"])-2460000.0, data_binA["MAG_median"], xerr=data_binA["eMJD"], yerr=data_binA["eMAG"], elinewidth=0.75, linewidth=0, ms=10,  marker=".", mfc="black",mec="black", ecolor="black")#, label = f"$σA _i = \Gamma ^{2}_i $ x {np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))}") # ({np.mean(sfA)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))
                            axis[i][j].errorbar((data_binB["MJD_central"])-2460000.0, data_binB["MAG_median"], xerr=data_binB["eMJD"], yerr=data_binB["eMAG"], elinewidth=0.75, linewidth=0, ms=10,  marker=".", mfc="black",mec="black", ecolor="black")
                            
                            axis[i][j].locator_params(axis='x', nbins=50)
                            
                            axis[i][j].legend()
                            axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
                            axis[i][j].set_title("Filter "+unique_values[i] + " comp. "+ diff_phot_names_compA[t]['filter_' + unique_values[i]][g][0]+ " and " + diff_phot_names_compB[t]['filter_' + unique_values[i]][g][0]+ " and ref. star " + diff_phot_names_compA[t]['filter_' + unique_values[i]][g][1:] +", $\Gamma ^2 $ using "+ l+ " and "+ (scale_factors_namesB[t][filtre][j]) +", σ_stars_"+ n )
    
    # plt.savefig('ejemplo'+'.jpeg')
    break
    # plt.savefig('diff_phot_components_error_binning'+str(t)+'.jpeg')
    